# S-metaheuristike

In [33]:
import random
from copy import deepcopy


## Pomoćne funkcije

In [34]:
def is_feasibel(solution):
    return any(solution)


## Inicijalizacija resenja

Jedno rešenje možemo predstaviti na dva načina:
- Kao skup pozicija na kojima se u rešneju nalazi jedinica.
- Kao niz logičkih vrednosti čija je dužina jednaka broju resursa i gde logičke vrednosti predstavljju da li je resurs u upotrebi ili niej.

U ovom zadatku koristimo drugu reprezentaciju


In [35]:

def initialize(num_resources):

    # Generišemo jedno nasumično rešenje
    solution = [random.random() < 0.25 for _ in range(num_resources)]

    # Pravimo nasumičnu malu izmenu ako generisano rešenje nije validno
    if not is_feasibel(solution):
        random_resourse = random.randrange(num_resources)
        solution[random_resourse] = True

    return solution


Funckiji koju računa vrednost trenutgno rešenja treba da prosledimo matricu sa cenama za upotrenu svakog resursa za svakog korisnika i niz s cenama za upotrebu svkog od resursa.

## Funkcija cilja

In [36]:
# cost.shape = (broj_korisnika, broj_resursa)
# fixes_cost.shape = (num_resources,)
def calc_solution_value(solution, cost_per_user, cost_per_resource):

    value = 0

    # Za svakog korisnika racunamo najjeftiniji resur u upotrebi
    # Minimalne cene sabiramo sa konacnim rezultatom
    for user_id, cost_for_user in enumerate(cost_per_user):

        min_cost_for_user = float('inf')

        for resource_id, resource_used in enumerate(solution):

            if resource_used is True and cost_for_user[resource_id] < min_cost_for_user:
                min_cost_for_user = cost_for_user[resource_id]

        value += min_cost_for_user

    # Cenu svakog resursa u upotrebi sabiramo sa konacnim rezultatom 
    for resource_id, resource_used in enumerate(solution):
        if resource_used:
            value += cost_per_resource[resource_id]

    return value


In [37]:
def small_change(solution):

    num_resources = len(solution)
    chosen_resource = random.randrange(0, num_resources)
    solution[chosen_resource] = not solution[chosen_resource]

    if not is_feasibel(solution):
        solution[chosen_resource] = not solution[chosen_resource]
        return -1
    else:
        return chosen_resource


In [38]:
def read_instance(file_path):
    with open(file_path, 'r') as f:
        num_users, num_resources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()]
                for i in range(num_users)]
        fixed_cost = [int(x) for x in f.readline().split()]
        return cost, fixed_cost


## Lokalna pretraga

In [39]:
def local_serach(max_iters, costs_per_user, cost_per_resource):

    num_resources = len(cost_per_resource)

    solution = initialize(num_resources)
    curr_value = calc_solution_value(solution, costs_per_user, cost_per_resource)

    for i in range(max_iters):

        resource_inverted = small_change(solution)

        if resource_inverted < 0:
            continue

        new_value = calc_solution_value(solution, costs_per_user, cost_per_resource)

        if new_value < curr_value:
            curr_value = new_value
        else:
            solution[resource_inverted] = not solution[resource_inverted]

    return solution, curr_value


### Primer upotrebe

In [40]:
cost, fixed_cost = read_instance('data/uflp1.txt')
local_serach(100, cost, fixed_cost)


([False, True, True], 41)

In [41]:
local_serach(100, cost, fixed_cost)

([True, False, False], 34)

U ovom primeru, dolazimo do dva globalna minimuma:
1. Koristimo samo prvi resurs i dobijamo ukupnu vrednost 34
2. Koristimo drugi i treći resur i dobijamo ukupnu vrednost 41
Prvi lokalni minimum je ujedno i globani minimum.

## Simulirano kaljenje

Algoritam simuliranog kaljenja predstavlja modifikaciju algoritma lokalne pretrage. Razliku je se u tome što u slučaju kada smo naišli na suseda koji nema manju vrednost funkcije cilja, sa zadatom verovatnoćom ipak prelazimo na tog suseda. Verovatnoća prelaska na lošijeg suseda treba da bude takva da što je više iteracija prošlo, to je manja verovatnoća da pređemo na njega. Ovim je omogućeno da u ranim iteracijama pretrage imamo verovatnoću da skočimo u neku okolinu koja će nas dovesti do boljeg lokalnog minimuma.

In [42]:
def simulated_annealing(max_iters, costs_per_user, cost_per_resource):

    num_resources = len(cost_per_resource)
    
    solution = initialize(num_resources)
    curr_value = calc_solution_value(solution, costs_per_user, cost_per_resource)

    for i in range(1, max_iters):
        
        inverted_resource = small_change(solution)
        if inverted_resource < 0:
            continue

        new_value = calc_solution_value(solution, costs_per_user, cost_per_resource)

        if new_value < curr_value:
            curr_value = new_value
        else:
            p = 1.0 / i ** 0.5

            q = random.uniform(0, 1)

            if q < p:  # Prihvatamo losije resenje
                curr_value = new_value
            else:
                solution[inverted_resource] = not solution[inverted_resource]

    return solution, curr_value


### Primer upotrebe

In [43]:
cost, fixed_cost = read_instance('data/uflp1.txt')

In [44]:
simulated_annealing(100, cost, fixed_cost)

([True, False, False], 34)

In [45]:
simulated_annealing(100, cost, fixed_cost)

([True, False, False], 34)

In [46]:
simulated_annealing(100, cost, fixed_cost)

([True, False, False], 34)

In [47]:
simulated_annealing(100, cost, fixed_cost)

([True, False, False], 34)

Vidimo da kod simuliranog kaljenja možemo da dođemo do rešenja koji ne predstavljaju lokalne minimume. Ova pojava se dešava zbog nedeterminizma koji smo uveli.

In [80]:
def shaking(solution, k):

    num_resources = len(solution)

    chosen_resources = random.sample(range(num_resources), k)
    for resource in chosen_resources:
        solution[resource] = not solution[resource]
    
    if not is_feasibel(solution):
        random_resource = random.randrange(0, num_resources)
        solution[random_resource] = not solution[random_resource]
        chosen_resources = [random_resource]

    return chosen_resources

In [81]:
def vns(max_iters, k_max, costs_per_user, move_prob, cost_per_resource):

    num_resources = len(cost_per_resource)

    solution = initialize(num_resources)
    curr_value = calc_solution_value(solution, costs_per_user, cost_per_resource)

    for i in range(max_iters):
        for k in range(1, k_max):

            inverted_resources = shaking(solution, k)
            # new_solution = local_serach(solution)
            new_value = calc_solution_value(solution, costs_per_user, cost_per_resource)

            if new_value < curr_value or (new_value == curr_value and move_prob):
                curr_value = new_value
                break
            else:
                for resource in inverted_resources:
                    solution[resource] = not solution[resource]
                

    return solution, curr_value


In [82]:
cost, fixed_cost = read_instance('data/uflp1.txt')

In [116]:
vns(max_iters=10, k_max=2, move_prob=0.5, costs_per_user=cost, cost_per_resource=fixed_cost)

([False, True, True], 41)